In [71]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import pandas as pd
import numpy as np
from PIL import Image
from matplotlib import pyplot
from math import floor
import matplotlib.pyplot as plt

In [72]:
NUM_INPUTS = 15
NUM_OUTPUTS = 11

# Load the dataset input and output
inputNums = np.loadtxt('../dataset/0_input.csv', delimiter=',') # (num training points, 13)
y = np.loadtxt('../dataset/0_output.csv', delimiter=',') # (num training points, 11)
X = np.zeros((len(inputNums), 480, 640)) # (num training points, 480, 640)

# For each of the training examples there should be exactly 1 image
for i in range(0, len(inputNums)):
    image = Image.open('../dataset/img_' + str(i) + '.jpg') 
    imageArr = np.asarray(image) # Converts the image into a 2D 480*640 array
    X[i] = imageArr / 255


In [108]:
numerical_inputs = Input(shape=(NUM_INPUTS,))
image_inputs = Input(shape=(480, 640, 1))

# Process the image data first
x_image = Conv2D(32, (7, 7), strides=(4, 4), padding='same', activation='relu')(image_inputs)
x_image = Conv2D(64, (5, 5), strides=(4, 4), padding="same", activation="relu")(x_image)
x_image = Conv2D(128, (3, 3), strides=(4, 4), padding="same", activation="relu")(x_image)
x_image = Conv2D(256, (3, 3), strides=(4, 4), padding="same", activation="relu")(x_image)
x_image = Model(image_inputs, x_image)
x_image = Flatten()(x_image.output)

print(x_image.shape)
print(numerical_inputs.shape)

# Combine the latent space representation with the numerical input
combined = concatenate([x_image, numerical_inputs])
print(combined.shape)
x_combined = Dense(300, activation='relu')(combined)
x_combined = Dense(60, activation='relu')(x_combined)
outputs = Dense(11, activation='linear')(x_combined)

model = Model([image_inputs, numerical_inputs], outputs)

model.compile(loss='mse', optimizer='adam', metrics=['mae'])
history = model.fit(epochs=1, batch_size=32, validation_split=0.2)

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

(None, 1536)
(None, 15)
(None, 1551)


ValueError: Failed to find data adapter that can handle input: <type 'NoneType'>, <type 'NoneType'>